# Optymalizacja zapytań django orm

Pracować będziemy na modelu postaci:

<code>
class Product(models.Model):
    title = models.CharField(max_length=100)
    manufacturer = models.CharField(max_length=100)
    price = models.IntegerField()
</code>
<code>
    a = models.CharField(max_length=100)
    b = models.CharField(max_length=100)
    c = models.CharField(max_length=100)
</code>
<code>    
    product_secret_id = models.CharField(max_length=100)
</code>

Dopisz powyższy model do jednej ze swoich aplikacji.

Najpierw linijki, które pozwolą nam swobodnie korzystać z Django w notatniku Jupyter.

In [4]:
import os
import django
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'rest.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

Zaimportujmy model

In [5]:
from relations.models import Product

Dodajmy kilka wpisów

In [17]:
product1 = Product.objects.create(
    title='test1',
    manufacturer='test',
    price=100,
    a='ala',
    b='ma',
    c='kota',
    product_secret_id='1111'
)

product2 = Product.objects.create(
    title='test2',
    manufacturer='test',
    price=10,
    a='kot',
    b='ma',
    c='ale',
    product_secret_id='2222'
)

product3 = Product.objects.create(
    title='test3',
    manufacturer='test',
    price=1,
    a='ewa',
    b='ma',
    c='psa',
    product_secret_id='3333'
)

<Product: Product object (3)>

I napiszmy zapytanie o wszystkie wpisy w tabelce.

In [34]:
products = Product.objects.all()
print(products.query)

SELECT "relations_product"."id", "relations_product"."title", "relations_product"."manufacturer", "relations_product"."price", "relations_product"."a", "relations_product"."b", "relations_product"."c", "relations_product"."product_secret_id" FROM "relations_product"


Powyższe zapytanie zwraca nam wartości WSZYSTKICH KOLUMN ze wszystkich wpisów w tabelce. Jak napisać zapytanie, które wyciągnie wartości tylko z wybranych kolumn?

Możemy oczywiście wyciągnąć z queryseta tylko te wartości, które nas interesują

In [35]:
[(product.title, product.manufacturer, product.price) for product in products]

[('test1', 'test', 100), ('test2', 'test', 10), ('test3', 'test', 1)]

Ale w ten sposób wciąż wyciągamy z bazy znacznie więcej informacji niż jest nam potrzebne (i dopiero w pythonie wybieramy z tego wyciąganiętego zbiory tylko te informacje, które nas interesują). Może to generować szereg poważnych problemów:
- im więcej danych chcemy wyciągnąć z bazy, tym dłużej użytkownicy będą czakać na wykonanie zapytania (oraz przesyłanie wyciągniętych danych) 
- w czasie kiedy wykonywane jest zapytanie na bazie inne zapytania muszą czekać, co ogranicza liczbę użytkowników jaką nasza aplikacja może płynnie obsłużyc
- obecnie, często w rozwiązaniach chmurowych za wykonanie zapytania płaci się proporcjonalnie do czasu wykonywania tego zapytania na bazie, co jeszcze bardziej podnosi rangę problemu

Oczywistym staje się potrzeba zminimalizowania ilość danych przesyłanych pomiędzy bazą a aplikacją. Chcemy wyciągnąć z bazy wyłącznie te informacje, które są nam potrzebne (co zawsze jest dobrą praktyką).

Mamy 3 metody, których możemy w tym celu użyć:
- `only`
- `values`
- `defer`

### only

In [19]:
products = Product.objects.all().only('title', 'manufacturer', 'price')
print(products.query)

SELECT "relations_product"."id", "relations_product"."title", "relations_product"."manufacturer", "relations_product"."price" FROM "relations_product"


In [20]:
print(products)

<QuerySet [<Product: Product object (1)>, <Product: Product object (2)>, <Product: Product object (3)>]>


W wyniku otrzymujemy tradycyjnego queryseta, którego wartości to obiekty klasy Product

Użycie only nie zabezpiecza nas jednak przed odpytywaniem bazy o pola, których nie wyciągneliśmy z bazy. W przypadku, gdybyśmy odwołali się do pola a wpisu, django orm po prostu drugi raz uderzy na baze i pobierze tę wartość.

In [23]:
products[0].a

'ala'

W skrajnych przypadkach może to prowadzić do wykonania setek/tysięcy dodatkowych zapytań na bazę. Dlatego należy zwrócić szczególną uwagę na to, żeby w metodzie only umieścić wszystkie pola modelu, które zamierzamy użyć.

Do dyspozycji mamt też drugą metodą - `values`, która w niektórych scenariuszach moze okazać się lepszym rozwiązaniem.

### values

In [24]:
products = Product.objects.all().values('title', 'manufacturer', 'price')
print(products.query)

SELECT "relations_product"."title", "relations_product"."manufacturer", "relations_product"."price" FROM "relations_product"


In [25]:
print(products)

<QuerySet [{'title': 'test1', 'manufacturer': 'test', 'price': 100}, {'title': 'test2', 'manufacturer': 'test', 'price': 10}, {'title': 'test3', 'manufacturer': 'test', 'price': 1}]>


Tym razem dostaliśmy queryset, ale słowników (a nie obiektów klasy Product). 

In [28]:
print(products[0])

{'title': 'test1', 'manufacturer': 'test', 'price': 100}


In [29]:
print(type(products[0]))

<class 'dict'>


Korzyści:

1. słowniki pythonowe zajmują mniej pamięci niż obiekty modelu
2. nie jesteśmy już w stanie z pojedynczego elementu tego queryseta wyciągnąć wartość pola, którego nie zawarliśmy w parametrach metody `values`. 

In [30]:
print(products[0]['title'])

test1


Istnieje jeszcze trzecia metoda, której możemy tutaj użyć - `defer`

### defer

In [31]:
# TODO